## 2. Report Generation

The final phase of SDMT involves aggregating evidence, validating the metrics reflected by the evidence we collected, and displaying this information in a report.

#### Preliminaries

In [1]:
# Preliminaries for loading the package locally
import os
import sys

def package_root() -> str:
    """Resolve the path to the project root."""
    return os.path.abspath(os.path.join(os.getcwd(), "..", "src/"))

sys.path.append(package_root())

#### Initialize MLTE Context

In [2]:
import mlte

store_path = os.path.join(os.getcwd(), "store")

mlte.set_model("IrisClassifier", "0.0.1")
mlte.set_artifact_store_uri(f"local://{store_path}")

#### Define a Binding

In MLTE, a `Binding` associates individual results with the properties to which they attest.

In [3]:
from mlte.binding import Binding

binding = Binding(
    {
        "TaskEfficacy": [
            "accuracy",
            "confusion matrix",
            "class distribution"
        ],
        "StorageCost": [
            "model size"
        ],
        "TrainingComputeCost": [
            "training cpu"
        ],
        "TrainingMemoryCost": [
            "training memory"
        ]
    }
)

# Binding also supports persistence
binding.save()

#### Load Specification

In [4]:
from mlte.spec import Spec
from mlte.measurement.result import Real, Integer
from mlte.measurement.memory import MemoryStatistics

# Load the specification
spec = Spec.load()

#### Validate Results

In [5]:
from mlte.measurement.result import Integer
model_size: Integer = Integer.load("model size")
model_size = model_size.less_than(3000)

In [6]:
from mlte.measurement.cpu import CPUStatistics
cpu_utilization: CPUStatistics = CPUStatistics.load("training cpu")
cpu_utilization = cpu_utilization.max_utilization_less_than(5.0)

In [7]:
from mlte.measurement.memory import MemoryStatistics
memory_consumption: MemoryStatistics = MemoryStatistics.load("training memory")
memory_consumption = memory_consumption.average_consumption_less_than(1_000_000)

In [8]:
from mlte.measurement.result import Real
accuracy: Real = Real.load("accuracy")
accuracy = accuracy.greater_or_equal_to(0.9)

In [9]:
from mlte.measurement.result import Image
class_distribution: Image = Image.load("class distribution")
class_distribution = class_distribution.ignore("Inspect the image.")